In [1]:
import exiftool
from astropy.time import Time
import astropy.units as u
raw_path = '010A7115.CR3'
with exiftool.ExifToolHelper() as et:
    exif = et.get_metadata(raw_path)[0]
    time = exif['EXIF:DateTimeOriginal']
    offset = exif['EXIF:OffsetTime']
time = time.replace(':','-',2)
time = Time(time)-int(offset[0:3])*u.hour

In [6]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, angular_separation, position_angle, offset_by
bj = EarthLocation(lon=115*u.deg+14*u.arcsec,lat=41*u.deg+13*u.arcmin+53*u.arcsec,height = 1466)
dunhuang  = EarthLocation(lon=94.322799*u.deg,lat=40.359581*u.deg,height = 1100)
import numpy as np


In [5]:
def alt2pres(altitude):
    # https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/atmosphere.html
    press = 100 * ((44331.514 - altitude) / 11880.516) ** (1 / 0.1902632)

    return press/100*u.hPa

f1 = AltAz(obstime=Time.now(), location=bj, pressure = alt2pres(bj.height.value),temperature=0*u.deg_C,relative_humidity=0.5,obswl=550*u.nm)
f2 = AltAz(obstime=Time.now(), location=bj)
s = SkyCoord(ra=[14/24*360,9],dec=[45,8],frame='icrs', unit='deg')
s.transform_to(f1).alt-s.transform_to(f2).alt

<Angle [0.16218671, 0.0096167 ] deg>

In [23]:
e = SkyCoord(az=0.5*np.pi,alt=0,unit=u.rad,frame=f1)
n = SkyCoord(az=0,alt=40,unit= u.deg,frame=f1)
position_angle(e.az,e.alt,n.az,n.alt).to(u.deg)+50*u.deg

<Angle 360. deg>

In [14]:
s1 = s.transform_to(f1)

In [15]:
angular_separation(0*u.rad, 0*u.rad, s1.az, s1.alt)

<Quantity [0.10093712, 0.73137107] rad>

In [37]:
type(s.transform_to(f1))

astropy.coordinates.sky_coordinate.SkyCoord

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from astropy.io import fits
from astropy.table import Table, vstack
from matplotlib.collections import LineCollection
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, angular_separation, position_angle, offset_by
# from photutils.aperture import CircularAperture
from astropy.stats import sigma_clipped_stats
# from photutils.detection import find_peaks
import tetra3
import rawpy
import exiftool
from photutils.detection import find_peaks, DAOStarFinder
import astropy.units as u
from astropy.time import Time

from scipy.optimize import minimize, curve_fit, fsolve

def lens_func( theta, f, k, ks=[0]*5):
    # r = f*np.sin(k*theta)/k
    def func(r):
        return np.arcsin(r*k/f)/k-(ks[0]*r**5 + ks[1]*r**4 + ks[2]*r**3 + ks[3]*r**2 + ks[4]*r)-theta

    return fsolve(func, 10)[0]
lens_func(0.4,14.6,-0.39)

5.8163417656954

In [3]:
([0]*5)[4]

0